In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
# import seaborn as sns
import os

# Load data 

In [2]:
neoplasm_trn = 'abstrct-master/AbstRCT_corpus/data/train/neoplasm_train/'
neoplasm_tst = 'abstrct-master/AbstRCT_corpus/data/dev/neoplasm_dev/'
neoplasm_dv = 'abstrct-master/AbstRCT_corpus/data/test/neoplasm_test/'
mixed = 'abstrct-master/AbstRCT_corpus/data/test/mixed_test/'
glaucoma = 'abstrct-master/AbstRCT_corpus/data/test/glaucoma_test/'


# .txt files?? - not for now - the problem is with .ann data
def get_annotated_data(url):
    files = glob.glob(url+"/*.ann")
# print(len(files))
    
    df = pd.DataFrame()
    for f in files:

        df_temp = pd.read_csv(f, sep='\t', header = None)
        df = df.append(df_temp)
        df['url'] = f[50:-4]
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
#     print(df.info())
    return df
# df.head()

# Neoplasm
neoplasm_train = get_annotated_data(neoplasm_trn)
neoplasm_test = get_annotated_data(neoplasm_tst)
neoplasm_dev = get_annotated_data(neoplasm_dv)

# Glaucoma
glaucoma_test = get_annotated_data(glaucoma)

# Mixed
mixed_test = get_annotated_data(mixed)

In [3]:
neoplasm_train.head()

,index,0,1,2,url
0,0,T1,Premise 761 825,The overall remission rate was 87% with 31% co...,train/15920748
1,1,T2,Premise 826 932,The median survival of all 406 eligible patien...,train/15920748
2,2,T3,Premise 933 1107,"The overall remission rate, the rate of comple...",train/15920748
3,3,T4,Premise 1108 1248,In limited disease the estimated percentages o...,train/15920748
4,4,T5,Premise 1249 1437,Patients with extensive disease survived signi...,train/15920748


In [4]:
glaucoma_test.head()

,index,0,1,2,url
0,0,T1,Premise 683 812,Percentage of IOP reduction or the magnitude o...,est/16899528
1,1,T2,Premise 813 982,"In the visual field, the estimated rate of cha...",est/16899528
2,2,T3,Premise 983 1130,The estimated rate of change in MD showed no s...,est/16899528
3,3,T4,Premise 1131 1292,No changes in the optic nerve head topography ...,est/16899528
4,4,T5,Premise 1293 1378,There were no patients who dropped out due to ...,est/16899528


In [5]:
# Neoplasm
# neoplasm_train.info() #  2267
# neoplasm_test.info() # 326
# neoplasm_dev.info() # 686

# Glaucoma
# glaucoma_test.info() # 594

# Mixed
mixed_test.info()  # 600

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   600 non-null    int64 
 1   0       600 non-null    object
 2   1       600 non-null    object
 3   2       600 non-null    object
 4   url     600 non-null    object
dtypes: int64(1), object(4)
memory usage: 23.6+ KB


In [6]:
neoplasm_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   686 non-null    int64 
 1   0       686 non-null    object
 2   1       686 non-null    object
 3   2       686 non-null    object
 4   url     686 non-null    object
dtypes: int64(1), object(4)
memory usage: 26.9+ KB


In [7]:
def frequencies(df):
    components_freq = {}
    for i in df[1]:
        comp = i.split()
        if comp[0] in components_freq:
            components_freq[comp[0]] += 1
        else:
            components_freq[comp[0]] = 1
    return components_freq
print("Neoplasm train: " , frequencies(neoplasm_train))
print("Neoplasm test: " , frequencies(neoplasm_test))
print("Neoplasm dev: ",frequencies(neoplasm_dev))
print('\n')
print("Glaucoma test: ",frequencies(glaucoma_test))
print('\n')
print('Mixed test: ', frequencies(mixed_test))

Neoplasm train:  {'Premise': 1537, 'Claim': 666, 'MajorClaim': 64}
Neoplasm test:  {'Premise': 218, 'Claim': 99, 'MajorClaim': 9}
Neoplasm dev:  {'Premise': 438, 'Claim': 228, 'MajorClaim': 20}


Glaucoma test:  {'Premise': 404, 'Claim': 183, 'MajorClaim': 7}


Mixed test:  {'Premise': 388, 'Claim': 182, 'MajorClaim': 30}


## Add column with relation componentonly

In [8]:
def relation_components(df):
    components = list(df[1])
    comp_list = []
    for c in components:
        comp_list.append(c.split()[0])
    df['relation_components'] = comp_list

    return df

neoplasm_train = relation_components(neoplasm_train)
neoplasm_test = relation_components(neoplasm_test)
neoplasm_dev = relation_components(neoplasm_dev)

glaucoma_test = relation_components(glaucoma_test)

mixed_test = relation_components(mixed_test)

In [9]:
mixed_test.head()

,index,0,1,2,url,relation_components
0,0,T1,Premise 763 1114,"In sildenafil versus placebo arms, week-12 6MW...",/28371482,Premise
1,1,T2,Premise 1188 1295,Changes in WHO functional class and Borg dyspn...,/28371482,Premise
2,2,T3,Premise 1296 1363,"Headache, diarrhoea, and flushing were more co...",/28371482,Premise
3,3,T4,Claim 1364 1497,"Sildenafil, in addition to stable (≥3 months) ...",/28371482,Claim
4,0,T1,Premise 1287 1583,Responses (> or = 50% improvement) were seen i...,/28371482,Premise


In [12]:
print('Mean number of symbols - NEOPLASM: ')
print("\t train set: ", int(neoplasm_train[2].apply(lambda x: len(x)).mean()))
print("\t test set: ", int(neoplasm_test[2].apply(lambda x: len(x)).mean()))
print("\t dev set: ", int(neoplasm_dev[2].apply(lambda x: len(x)).mean()))
print('\n')
print('Mean number of symbols - GLAUCOMA: ', int(glaucoma_test[2].apply(lambda x: len(x)).mean()))
print('\n')
print('Mean number of symbols - MIXED: ', int(mixed_test[2].apply(lambda x: len(x)).mean()))


Mean number of symbols - NEOPLASM: 
	 train set:  138
	 test set:  143
	 dev set:  135


Mean number of symbols - GLAUCOMA:  138


Mean number of symbols - MIXED:  145


In [14]:
print('Mean number of words - Neoplasm: ')
print("\t train set: ", int(neoplasm_train[2].apply(lambda x: len(x.split())).mean()))
print("\t test set: ", int(neoplasm_test[2].apply(lambda x: len(x.split())).mean()))
print("\t dev set: ", int(neoplasm_dev[2].apply(lambda x: len(x.split())).mean()))
print('\n')
print('Mean number of symbols - GLAUCOMA: ', int(glaucoma_test[2].apply(lambda x: len(x.split())).mean()))
# print("\t glaucoma_test: ", int(glaucoma_test[2].apply(lambda x: len(x.split())).mean()))
print('\n')
print('Mean number of symbols - MIXED: ', int(mixed_test[2].apply(lambda x: len(x.split())).mean()))
# print("\t mixed_test: ", int(mixed_test[2].apply(lambda x: len(x.split())).mean()))

Mean number of words - Neoplasm: 
	 train set:  21
	 test set:  22
	 dev set:  20


Mean number of symbols - GLAUCOMA:  21


Mean number of symbols - MIXED:  22


# Conll


#### Neoplasm

<!-- Glaucoma test:  {'Premise': 404, 'Claim': 183, 'MajorClaim': 7} -->


<!-- Mixed test:  {'Premise': 388, 'Claim': 182, 'MajorClaim': 30} -->

In [15]:
train_conll = pd.read_csv('ecai2020-transformer_based_am/data/neoplasm/train.conll', sep='\t', header=None)
test_conll = pd.read_csv('ecai2020-transformer_based_am/data/neoplasm/test.conll', sep='\t', header=None)
dev_conll = pd.read_csv('ecai2020-transformer_based_am/data/neoplasm/dev.conll', sep='\t', header=None)
train_conll.head()

,0,1,2,3,4
0,0,Facial,_,_,B-Claim
1,1,hirsutism,_,_,I-Claim
2,2,is,_,_,I-Claim
3,3,one,_,_,I-Claim
4,4,of,_,_,I-Claim


In [16]:
print("Train: \n", train_conll[4].value_counts())

Train: 
 O            61110
I-Premise    42438
I-Claim      14383
B-Premise     1535
B-Claim        729
Name: 4, dtype: int64


        Supposed to be : B-premise = 1537, B-claim = 730 ???

In [17]:
print("Test: \n",test_conll[4].value_counts())

Test: 
 O            16739
I-Premise    11931
I-Claim       4677
B-Premise      438
B-Claim        248
Name: 4, dtype: int64


* Test => **Premise**: 218, **Claim**: 108 (99 + 9 (MajorClaim)) 

Should be compeletly different numbers according to the files in (.txt and .ann). 

***Looks like in .conll test is dev in .ann and vice versa***

In [18]:
print("Dev: \n",dev_conll[4].value_counts())

Dev: 
 O            9124
I-Premise    6655
I-Claim      2087
B-Premise     218
B-Claim       108
Name: 4, dtype: int64


**YES!** (dev, test = test, dev)

* Dev => **Premise**: 438, **Claim**: 248 (228 + 20(**MajorClaim**))

______________
Dev => Premise: 218, Claim: 108 <br>
Test => Premise: 438, Claim: 248
______________


#### Glaucoma

In [21]:
gl_conll = pd.read_csv('ecai2020-transformer_based_am/data/glaucoma_test/test.conll', sep='\t', header=None)
gl_conll.head()

,0,1,2,3,4
0,1,The,_,_,O
1,2,aim,_,_,O
2,3,of,_,_,O
3,4,this,_,_,O
4,5,study,_,_,O


In [22]:
print("Glaucoma: \n", gl_conll[4].value_counts())

Glaucoma: 
 O            15919
I-Premise    11483
I-Claim       3357
B-Premise      404
B-Claim        190
Name: 4, dtype: int64


* Glaucoma test:  **Premise** -> 404; **Claim**: 190 (183 + 7 (MajorClaim))

#### Mixed

In [23]:
mx_conll = pd.read_csv('ecai2020-transformer_based_am/data/mixed_test/test.conll', sep='\t', header=None)
mx_conll.head()

,0,1,2,3,4
0,0,We,_,_,O
1,1,previously,_,_,O
2,2,reported,_,_,O
3,3,that,_,_,O
4,4,treatment,_,_,O


In [24]:
print("Mixed: \n", mx_conll[4].value_counts())

Mixed: 
 O            16015
I-Premise    11767
I-Claim       3942
B-Premise      388
B-Claim        212
Name: 4, dtype: int64


* Mixed test:  **Premise**: 388, **Claim** : 212 (182 + 30 (MajorClaim))

###### !!! Problem in neoplasm trian set 

In [25]:
def check(url):
    check_lines = []
    with open(url, 'r') as f:
        lines = f.readlines()
    #     last_line = lines[-2]
    # print(lines[0].split('\t')[:50])
    for i, line in enumerate(lines[:-1]):
        if line != '\n':
            _, tkn, _, _, tag = line.split('\t')
            
        elif line == '\n' and lines[i+1] != '\n' and lines[i+1].split('\t')[4].startswith('I-'):
            check_lines.append(i+1)
    return check_lines

In [28]:
train_check = check('ecai2020-transformer_based_am/data/neoplasm/train.conll')
train_check

[]

<!-- 
# train_conll[train_conll[0] == 279][train_conll[1] == '4'] 
# train_conll[train_conll[0] == 359][train_conll[1] == '0001'] # 10129	359	0001	_	_	I-Premise
# train_conll[train_conll[0] == 400][train_conll[1] == '0002'] # 10170	400	0002	_	_	I-Premise
# train_conll[train_conll[0] == 213][train_conll[1] == '93'] # 11510	213	93	_	_	I-Premise
# train_conll[train_conll[0] == 242][train_conll[1] == '01'] # 15080	242	01	_	_	I-Premise
# train_conll[train_conll[0] == 271][train_conll[1] == '004'] # 33253	271	004	_	_	I-Premise
# train_conll[train_conll[0] == 234][train_conll[1] == 'US'] # 34268	234	US	_	_	I-Premise
# train_conll[train_conll[0] == 213][train_conll[1] == '048'] # 48940	213	048	_	_	I-Premise
# train_conll[train_conll[0] == 189][train_conll[1] == 'pain'] # 51615	189	pain	_	_	I-Premise
# train_conll[train_conll[0] == 210][train_conll[1] == '12'] # 51951	210	12	_	_	I-Premise
# train_conll[train_conll[0] == 232][train_conll[1] == '7'] # 51973	232	7	_	_	I-Premise
# train_conll[train_conll[0] == 205][train_conll[1] == 'better'] # 92762	205	better	_	_	I-Premise
 -->

In [189]:
def find_lines(arr, data):
    '''
    find lines that are causing problems to find out why
    
    returns 10 tokens before and after the problematic line
    '''
    chunk = []
    with open(data) as f:
            lines = f.readlines()
    for idx in arr:
        start = idx - 10
        end = idx + 10
        print(start)
        c = ''
        for i, line in enumerate(lines): 
           
            if i >= start and i <= end:
                if line != '\n':
                    c += line.split('\t')[1] + ' '
        chunk.append(c)
                    
    return chunk
        
lines_to_check = find_lines(check_lines, 'ecai2020-transformer_based_am/data/neoplasm/train.conll')

10434
10517
10560
11948
15662
34565
35624
50891
53688
54037
54061
96489


In [190]:
lines_to_check

['95 % CI = -9.6 % to -1 . 4 % ) per 4 weeks in the control group but ',
 '=.0002 , P =.0001 , and P = . 0001 , respectively , for ATP versus no ATP ) . ',
 '-0.2 % versus -2.4 % ; P = . 0002 ) ; functional scores ( +0.4 % versus -5.5 % ',
 '; 95 % CI , 0.61 to 0 . 93 ; P = .008 ) . Median survival was ',
 'and nine symptoms ( .001 < P < . 01 ) , and the improvement ( > 10 units on ',
 'responders indicated better physical well-being ( P = . 004 ) and mood ( P =.02 ) at month 3 ',
 'patients than in the control group , i.e . US $ 40782 and US $ 34465 , respectively ( P ',
 'survival rate , 31.8 % ; P = . 048 ) . Comparing survival for the two dose levels ',
 'and resolved . Mean ( s.d . ) pain scores on the day of discharge were 1·9 ( 3·1 ',
 ', 16.2 v 14.7 months ; P = . 12 ) . LV5FU2 plus oxaliplatin gave higher frequencies of ',
 'common toxicity criteria grade 3/4 neutropenia ( 41 . 7 % v 5.3 % of patients ) , grade 3/4 ',
 'QOL ( p < 0.05 ) ( i.e . better global health status , an

**One sentence** - was splitted because empty space after '.'(dot). For example: ~". 001" 

'95 % CI = -9.6 % to -1 . 4 % ) per 4 weeks in the control group but ' => ***10675381.ann***   
'=.0002 , P =.0001 , and P = . 0001 , respectively , for ATP versus no ATP ) . ' => ***10675381.ann***   
'-0.2 % versus -2.4 % ; P = . 0002 ) ; functional scores ( +0.4 % versus -5.5 %'  => ***10675381.ann***   
'; 95 % CI , 0.61 to 0 . 93 ; P = .008 ) . Median survival was ' => ***15625369.ann***   
'and nine symptoms ( .001 < ***P < . 01***) , and the improvement ( > 10 units on ' => ***10561201.ann***   
responders indicated better physical well-being **( P = . 004 )** and mood ( P =.02 ) at month 3 => ***10561203.ann***   
patients than in the control group , i.e . US $ 40782 and US $ 34465 , respectively ( P => ***9531327.ann***   
survival rate , 31.8 % ; P = **. 048 )** . Comparing survival for the two dose levels => ***10653877.ann***   
and resolved . Mean ( s.d . ) pain scores on the day of discharge were 1·9 ( 3·1 => ***23254324.ann***   
, 16.2 v 14.7 months ; P = . 12 ) . LV5FU2 plus oxaliplatin gave higher frequencies of => ***10944126.ann***   
common toxicity criteria grade 3/4 neutropenia ( 41 . 7 % v 5.3 % of patients ) , grade 3/4 => ***10944126.ann***  
QOL ( p < 0.05 ) ( i.e . better global health status , and physical and emotional functioning => ***23866850.ann***  

In [196]:
# print(check_lines)

[10444, 10527, 10570, 11958, 15672, 34575, 35634, 50901, 53698, 54047, 54071, 96499]


#### Test and Dev

In [29]:
test = 'ecai2020-transformer_based_am/data/neoplasm/test.conll'
dev = 'ecai2020-transformer_based_am/data/neoplasm/dev.conll'

In [30]:
test_check = check(test)
dev_check = check(dev)

In [31]:
test_check

[1807, 1816, 1834, 1843, 1856, 1865, 33747]

In [214]:
dev_check

[]

In [216]:
tst_find_lines = find_wrong_lines(test_check, test)
tst_find_lines

1797
1806
1824
1833
1846
1855
33737


['both arms , with 12 % ( postop . RCT ) and 12 % ( pre-op . RCT ) ',
 'RCT ) and 12 % ( pre-op . RCT ) of patients , respectively , suffering from anastomotic leakage ',
 'from anastomotic leakage , 3 % ( postop . RCT ) and 3 % ( pre-op . RCT ) ',
 'RCT ) and 3 % ( pre-op . RCT ) from postoperative bleeding , and 6 % ( postop ',
 'postoperative bleeding , and 6 % ( postop . RCT ) and 4 % ( pre-op . RCT ) ',
 'RCT ) and 4 % ( pre-op . RCT ) from delayed wound healing . The patient accrual ',
 'the 6 mg ibandronate group . I.v . ibandronate treatment leads to significant improvements in quality of life , ']

In [217]:
find_wrong_lines(dev_check, dev)

[]

### Glaucoma

**Premise**: 404  
**Claim**: 183   
**MajorClaim**: 7  

In [206]:
glaucoma_conll = pd.read_csv('ecai2020-transformer_based_am/data/glaucoma_test/test.conll', sep='\t', header=None)
glaucoma_conll.head()

,0,1,2,3,4
0,1,The,_,_,O
1,2,aim,_,_,O
2,3,of,_,_,O
3,4,this,_,_,O
4,5,study,_,_,O


In [207]:
print("Glaucoma: \n", glaucoma_conll[4].value_counts())

Glaucoma: 
 O            15919
I-Premise    11483
I-Claim       3357
B-Premise      404
B-Claim        190
Name: 4, dtype: int64


In [208]:
glaucoma_check = check('ecai2020-transformer_based_am/data/glaucoma_test/test.conll')
glaucoma_check

[114, 18541, 24453, 31537]

In [209]:
glaucoma_incorrect_lines = find_wrong_lines(glaucoma_check, 'ecai2020-transformer_based_am/data/glaucoma_test/test.conll')
glaucoma_incorrect_lines

104
18531
24443
31527


['9.8 and 30.6 ± 9.9 mmHg , respectively . Postoperatively , mean IOPs were 11.4 ± 4.9 and 13.6 ± ',
 'least as effective as pilocarpine 2 % t.i.d . in reducing IOP when added to eyes currently on monotherapy with ',
 'the trabeculectomy group ( P = 0.7 ) . Mean postoperative IOP was 13.7+/-2.2 mmHg at 3 months , 14.8+/-3.3 ',
 ', 24.7 mm Hg in the medical group . Mean IOP ( both eyes ) at last follow-up was 18.2 ']

## Mixed

In [222]:
mixed = 'ecai2020-transformer_based_am/data/mixed_test/test.conll'
mixed_conll = pd.read_csv(mixed, sep='\t', header=None)
mixed_conll.head()

,0,1,2,3,4
0,0,We,_,_,O
1,1,previously,_,_,O
2,2,reported,_,_,O
3,3,that,_,_,O
4,4,treatment,_,_,O


In [223]:
mixed_conll[4].value_counts()

O            16015
I-Premise    11767
I-Claim       3942
B-Premise      388
B-Claim        212
Name: 4, dtype: int64

In [224]:
mixed_check = check(mixed)
mixed_check

[32920]

In [225]:
print(find_wrong_lines(mixed_check, mixed))


32910
['glucose levels < 54 mg/dL by 65 % . ( 3.1 vs. 8.9 min , P < 0.001 ) . ']


## Token classification results

<!-- ### Results from authors

<img src='./pics/result_authors_BERT.png'>
<img src='./pics/results_fine-tune.png'>
<img src='./pics/results_all.png'> -->

In [ ]:
# Prev - before finding the broken sequences

||Neoplasm||||Glaucoma||||Mixed|||
|-|-|-|-|-|-|-|-|-|-|-|-|
||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1|
|SciBert+GRU+CRF|.84|.77|.93||.84|.82|.92||.82|.77|.91|
|BioBert+GRU+CRF|.81|.74|.91||.84|.77|.94||.82|.78|.92|
|Bert+GRU+CRF|.82|.72|.91||.83|.80|.89||.83|.77|.90|
|||||||||||||
|SciBert+LSTM+CRF|.70|.73|.90||.70|.72|.92||.69|.74|.90|
|||||||||||||
|mBert+LSTM+CRF|.84|.76|.92|||||||||

In [ ]:
# New - after 'fixing' broken sequences in relation components

||Neoplasm||||Glaucoma||||Mixed|||
|-|-|-|-|-|-|-|-|-|-|-|-|
||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1|
|SciBert+GRU+CRF|.84|.76|.92||.86|.84|.92||.84|.77|.92|
|BioBert+GRU+CRF|.85|.76|.94||.88|.85|.92||.87|.81|.93|
|Bert+GRU+CRF|.84|.73|.93||.85|.80|.90||.84|.77|.90|
|mBert+GRU+CRF|.84|.74|.90||.83|.77|.85||.83|.74|.89|
|||||||||||||
|SciBert+LSTM+CRF|.84|.75|.93||.84|.83|.92||.82|.76|.91|

<!-- BioBert+LSTM+CRF|.83|.76|.93||.83|.82|.91||.82|.80|.92|
Bert+LSTM+CRF|.84|.75|.92||.82|.79|.88||.83|.77|.90|
mBert+LSTM+CRF|.83|.75|.89||.83|.76|.88||.83|.73|.90| -->

# Zero-shot

||Neoplasm||||Glaucoma||||Mixed|||
|-|-|-|-|-|-|-|-|-|-|-|-|
||||||**Simalign**|||||
||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1||F1-macro|C-F1|E-F1|
|mBERT+GRU+CRF|
|*DeepL*|-|-|-||.73|.65|.81||.74|.65|.84|
|*Opus*|.76|.70|.86||.74|.67|.81||.75|.69|.85|
||||||**Awesome**|||||
|*DeepL*|-|-|-||.69|.64|.80||.68|.63|.84|
|*Opus*|.70|.68|.85||.70|.66|.80||.69|.68|.84|

In [ ]:
# Simalign
# DeepL
# glaucoma
eval_f1_macro = 0.7330685163813109                                                         
eval_f1_micro = 0.821646355698744                                                          
f1_claim = 0.6463561440874156                                                              
f1_evidence = 0.8121752041573869

# mixed

eval_f1_macro = 0.7388845261294309             
eval_f1_micro = 0.8367470877270469             
f1_claim = 0.6476654253795474                  
f1_evidence = 0.8440443213296399

# Opus

eval_f1_macro = 0.7644822428729099 
eval_f1_micro = 0.8553562123129606 
f1_claim = 0.6960284145947692      
f1_evidence = 0.8603749351643737

# glaucoma
eval_f1_macro = 0.7441223628398054 
eval_f1_micro = 0.8188750370745989      
f1_claim = 0.6680855001387989  
f1_evidence = 0.8086816720257235 

# mixed

eval_f1_macro = 0.752125852927327      
eval_f1_micro = 0.8415440553116934     
f1_claim = 0.6909506463824012          
f1_evidence = 0.8451670926820032 

# Awesome
#Deepl

#Mixed
eval_f1_macro = 0.6813619797048373
eval_f1_micro = 0.8318150867700914   
f1_claim = 0.6320421265238405          
f1_evidence = 0.8376305332224379 
        
# glaucoma
eval_f1_macro = 0.6882386736630343    
eval_f1_micro = 0.8157818555353225    
f1_claim = 0.6379695053923391         
f1_evidence = 0.8032577722961766

# Opus

#mixed
eval_f1_macro = 0.6939731804602616                
eval_f1_micro = 0.8354966937322206                
f1_claim = 0.677382319173364                                 
f1_evidence = 0.8370890459699225 

# glaucoma
eval_f1_macro = 0.6971031397931412       
eval_f1_micro = 0.813957270926817                               
f1_claim = 0.658987459359034                                    
f1_evidence = 0.8009396206716548

#neoplasm
eval_f1_macro = 0.7028230781958845                   
eval_f1_micro = 0.8476699173676766                   
f1_claim = 0.6802940417848853                        
f1_evidence = 0.848349466368825


In [ ]:
# # SciBERT
# # Neoplasm
# eval_f1_macro = 0.8386974682145057                                   
# eval_f1_micro = 0.908901755020807                                    
# f1_claim = 0.7583465818759936                                        
# f1_evidence = 0.9232906848839229    

# #mixed

# eval_f1_macro = 0.841540512438424       
# eval_f1_micro = 0.9116866522508354      
# f1_claim = 0.774116237267825            
# f1_evidence = 0.9224381023209693 

# #Glaucoma

# eval_f1_macro = 0.8551391118626888    
# eval_f1_micro = 0.9215724385918025    
# f1_claim = 0.8381056069678824         
# f1_evidence = 0.9183874709976799 


# # BioBERT

# # Neoplasm 
# eval_f1_macro = 0.8549175952179474    
# eval_f1_micro = 0.9205766082338378    
# f1_claim = 0.7593402894648267         
# f1_evidence = 0.9420771952056604  

# #Glaucoma
# eval_f1_macro = 0.8804954993112277                         
# eval_f1_micro = 0.925114641764073                          
# f1_claim = 0.8520857422396159                              
# f1_evidence = 0.9179002624671916

# # Mixed

# eval_f1_macro = 0.867429705150827        
# eval_f1_micro = 0.9209362096570322      
# f1_claim = 0.8112363274776268                   
# f1_evidence = 0.9252932247826686   


# # BERT

# # Neoplasm
# eval_f1_macro = 0.8354437186845217        
# eval_f1_micro = 0.9010616953686674        
# f1_claim = 0.7348221670802315             
# f1_evidence = 0.9284487058693123

# # Mixed

# eval_f1_macro = 0.8449616853927541     
# eval_f1_micro = 0.9000343981979783     
# f1_claim = 0.7721152207260951          
# f1_evidence = 0.9035696793338903  

# # Glaucoma

# eval_f1_macro = 0.8452278803658868     
# eval_f1_micro = 0.9027378545275697     
# f1_claim = 0.7998094330633635          
# f1_evidence = 0.8968083127375163 

# # mBERT


# # Neoplasm
# eval_f1_macro = 0.8396294626651172
# eval_f1_micro = 0.8851769464105157
# f1_claim = 0.7405164169588777    
# f1_evidence = 0.9030980790839531 

# # Glaucoma
# eval_f1_macro = 0.8311648001332655        
# eval_f1_micro = 0.8659723115168659        
# f1_claim = 0.7743055555555556             
# f1_evidence = 0.8501562787277073

# # Mixed
# eval_f1_macro = 0.8255204466482781                                           
# eval_f1_micro = 0.8847666378565255             
# f1_claim = 0.741614214006852                   
# f1_evidence = 0.8899402951447561 




In [ ]:
# --------Prev---------
# # SciBERT

# #Neoplasm

# eval_f1_macro = 0.7034976018232387                    
# eval_f1_micro = 0.8956501420437076                    
# f1_claim = 0.7287622439893144                         
# f1_evidence = 0.9063168310511028  

# #Mixed

# eval_f1_macro = 0.693848607690201                                            
# eval_f1_micro = 0.895664988333538                                            
# f1_claim = 0.7351050561258755                                                
# f1_evidence = 0.904653312788906  

# #Glaucoma
# eval_f1_macro = 0.7029332753067948                                              
# eval_f1_micro = 0.8993804549360104                                              
# f1_claim = 0.7180550320392008                                                   
# f1_evidence = 0.9219492641585065  


# #mBERT
 
# # Neoplasm
# eval_f1_macro = 0.7034976018232387
# eval_f1_micro = 0.8956501420437076
# f1_claim = 0.7287622439893144
# f1_evidence = 0.9063168310511028

# #Mixed
# eval_f1_macro = 0.693848607690201
# eval_f1_micro = 0.895664988333538
# f1_claim = 0.7351050561258755
# f1_evidence = 0.904653312788906

# # Glaucoma
# eval_f1_macro = 0.6935133045360525
# eval_f1_micro = 0.9008886514835138
# f1_claim = 0.7631375152118596
# f1_evidence = 0.8985923585176673


# #mBert + GRU
# #Neoplasm
# eval_f1_macro = 0.8515126074714842
# eval_f1_micro = 0.9061463809138698
# f1_claim = 0.7587429492344883
# f1_evidence = 0.9302527389087754

# # Glaucoma
# eval_f1_macro = 0.844534075497075
# eval_f1_micro = 0.8890910647771546
# f1_claim = 0.7966230316827927
# f1_evidence = 0.8722666593868582

# # Mixed
# eval_f1_macro = 0.8371631163105483
# eval_f1_micro = 0.8970581882586721
# f1_claim = 0.7504187604690117
# f1_evidence = 0.9030445501978097


## Translations and Projections



* DeepL
* Opus
* Mixed


### IAA

In [34]:
import sklearn
from sklearn.metrics import cohen_kappa_score

# ann1 = [5,3,5,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,1,3,3,3,3,1,3,3]
# ann2 =[3,3,1,3,3,3,3,3,5,3,3,5,3,3,2,3,1=3,3,3,5,3,5,3=5,3,3,3,1=2=4=5,3,3]


ann1 = [2,3,5,3,3,3,3,3,1,3,3,3,3,3,3,3,3,3,3,3,3,1,3,3,3,3,1,3,3]
ann2 = [3,3,1,3,3,3,3,3,5,3,3,5,3,3,2,3,1,3,3,5,3,5,3,3,3,3,1,3,3]
print(cohen_kappa_score(ann1, ann2))



0.2817337461300309


#### Neoplasm 
   * train - 4405 lines
   * test - 1251 lines
   * dev - 679 lines

#### Glaucoma
   * test - 1247 lines

#### Mixed
   * test - 1148 lines
   

||train|dev|neoplasm|glaucoma|mixed|
|-|-|-|-|-|-|
|**DeepL**|✅|✅|-|✅|✅|
|**Opus-mt**|✅|✅|✅|✅|✅|
|**m2m_100**||✅|✅|-|-|



## Projections

||train|dev|neoplasm|glaucoma|mixed|
|-|-|-|-|-|-|
|**DeepL**|
|simalign|-|-|-|✅|✅|
|awesome|-|-|-|✅|✅|
|**Opus**|
|simalign|-|-|✅|✅|✅|
|awesome|-|-|✅|✅|✅|
|**m2m_100**|
|simalign|-|-|-|-|-|
|awesome|-|-|-|-|-|


